In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import nibabel as nib
from os import listdir
from os.path import isfile, join
from scipy import signal
import sys

# current path
cur_path = os.getcwd()
# file name and path
file = "BB_003_3T_cmrr_mbep2d_diff_0pt2_1_task_18s_off_12s_on_20200710180104_8"
file_name = cur_path + "/" + str(file) + "/dfMRI_raw.nii.gz"
# loading nifti object/header
raw_data_ = nib.load(file_name)
header  = raw_data_.header
raw_data = raw_data_.get_fdata()

### 1. Motion correction
#### 1.1 Split b0 and b1 values

In [29]:
def gzip(filename, zip):
    """
    Function that zips or unzips the filename file
    Input:
        filename : str
        zip : bool, true -> zip, false -> unzip
    Ouput:
        new_filename : str
    """
    if zip == False:
        new_filename = filename.replace('.gz', '')
        os.system(f"gzip -f -c -d {filename} > {new_filename}")
    elif zip == True:
        os.system(f"gzip {filename} -f")
        new_filename = f'{filename}.gz'
    else:
        new_filename = None
        
    return new_filename


In [30]:
import nipype.interfaces.spm as spm
import zipfile

dfmri_degibbs_sdc = nib.load(f'{cur_path}/data/topup/dfmri_degibbs_sdc.nii.gz')
dfmri_degibbs_sdc = dfmri_degibbs_sdc.get_fdata()
for i in range(2):
    dfmri_degibbs_sdc_b = dfmri_degibbs_sdc[...,np.arange(i, dfmri_degibbs_sdc.shape[3], 2)]
    img_deggibs_sdc_b = nib.Nifti1Image(dfmri_degibbs_sdc_b, np.eye(4))
    nib.save(img_deggibs_sdc_b, f'{cur_path}/data/ADC/dfmri_degibbs_sdc_b{i}.nii.gz') 

    gzip(f'{cur_path}/data/ADC/dfmri_degibbs_sdc_b{i}.nii.gz', zip=False)
 

In [27]:
# Matlab is needed
realign = spm.Realign()
print(type(f"{cur_path}/data/ADC/dfmri_degibbs_sdc_b0.nii.gz"))
realign.inputs.in_files = f"{cur_path}/data/ADC/dfmri_degibbs_sdc_b0.nii"
realign.run()

<class 'str'>


TraitError: Each element of the 'in_files' trait of a RealignInputSpec instance must be a pathlike object or string representing an existing file or a list of items which are a pathlike object or string representing an existing file, but a value of 'dfmri_degibbs_sdc_b0.nii.gz' <class 'str'> was specified.

In [32]:
# Since I don't have matlab, I take Wictor's output here
rdfMRI_sdc_b0 = nib.load('/home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_0.nii.gz')
rdfMRI_sdc_b0 = rdfMRI_sdc_b0.get_fdata()
rdfMRI_sdc_b1 = nib.load('/home/localadmin/Documents/preprocessing/data/ADC/rdfMRI_denoised_unringed_sdc_b_1.nii.gz')
rdfMRI_sdc_b1 = rdfMRI_sdc_b1.get_fdata()

rdfMRI_sdc_b0_mean = np.mean(rdfMRI_sdc_b0, axis=3)
rdfMRI_sdc_b1_mean = np.mean(rdfMRI_sdc_b1, axis=3)

(116, 116, 14)